### 读取数据

In [2]:
import pandas as pd

In [3]:
zl_order_data=pd.read_csv(r'C:/TM1py/order_lsku_ne.csv')

In [4]:
zl_order_data=zl_order_data[zl_order_data['yyyymm'].str.slice(0,4).astype('int')>2020].drop('yyyymm',axis=1)

In [5]:
zl_order_data.reset_index(drop=True,inplace=True)

In [6]:
zl_order_data

,order_id,lsku
0,026-0000462-1540302,LSKU2006
1,026-0001924-2492333,LSKU1933
2,026-0002258-7754707,LSKU1088
3,026-0003043-4180363,LSKU1149
4,026-0003702-5159525,LSKU1220
...,...,...
1507682,S03-9826401-6865450,LSKU1175
1507683,S03-9826401-6865450,LSKU1253
1507684,S03-9826401-6865450,LSKU1261
1507685,S03-9826401-6865450,LSKU1473


In [7]:
zl_order_multi_order=zl_order_data[zl_order_data.groupby('order_id')['order_id'].transform('count')>1]

In [8]:
zl_order_multi_order

,order_id,lsku
23,026-0047611-2829146,LSKU1071
24,026-0047611-2829146,LSKU1095
43,026-0106843-3116335,LSKU1860
44,026-0106843-3116335,LSKU1897
81,026-0192418-6945155,LSKU1180
...,...,...
1507679,S03-9725873-8674213,LSKU1473
1507682,S03-9826401-6865450,LSKU1175
1507683,S03-9826401-6865450,LSKU1253
1507684,S03-9826401-6865450,LSKU1261


In [9]:
zl_order_multi_order_drop=zl_order_multi_order.drop_duplicates()

In [10]:
zl_order_multi_order_drop

,order_id,lsku
23,026-0047611-2829146,LSKU1071
24,026-0047611-2829146,LSKU1095
43,026-0106843-3116335,LSKU1860
44,026-0106843-3116335,LSKU1897
81,026-0192418-6945155,LSKU1180
...,...,...
1507679,S03-9725873-8674213,LSKU1473
1507682,S03-9826401-6865450,LSKU1175
1507683,S03-9826401-6865450,LSKU1253
1507684,S03-9826401-6865450,LSKU1261


In [11]:
zl_order_multi_order_drop_index=zl_order_multi_order_drop.set_index('order_id')

In [12]:
zl_order_multi_order_drop_index

,lsku
order_id,
026-0047611-2829146,LSKU1071
026-0047611-2829146,LSKU1095
026-0106843-3116335,LSKU1860
026-0106843-3116335,LSKU1897
026-0192418-6945155,LSKU1180
...,...
S03-9725873-8674213,LSKU1473
S03-9826401-6865450,LSKU1175
S03-9826401-6865450,LSKU1253


#### 生成候选项集

In [13]:
order_id_unique=zl_order_multi_order_drop_index.index.unique()

In [15]:
def create_cl(dataset,order_id_list):
    cl=[]
    for order_id in order_id_list:
        transaction=dataset.loc[order_id]
        for item in transaction:
            if not {item} in cl:
                cl.append({item})
    cl.sort()
    return list(map(frozenset,cl))

#### 生成频繁项集

In [16]:
def create_freq_transaction(dataset,order_id_list,ck,min_support=0.5):
    sscnt={}
    for order_id in order_id_list:
        transaction=dataset.loc[order_id]
        for can in ck:
            if  can.issubset(transaction):
                sscnt[can]=sscnt.get(can,0)+1
    num_transactions=float(len(order_id_list))
    freq_transaction=[]
    support_data={}
    for key in sscnt:
        support=sscnt[key]/num_transactions
        if support>=min_support:
            support_data[key]=support
            freq_transaction.append(key)
    return support_data,freq_transaction

#### 生成第Ck个候选集

In [17]:
def create_ck(freq_k_transaction):
    ck=[]
    k=len(freq_k_transaction)
    for i in range(k):
        for j in range(i+1,k):
            t1=freq_k_transaction[i]
            t2=freq_k_transaction[j]
            t=t1|t2
            if(not t in ck)and(len(t)==len(freq_k_transaction[0])+1):
                ck.append(t)
    return ck             

#### Apriori函数

In [18]:
def Apriori(dataset,order_id_list,min_support=0.5):
    cl=create_cl(dataset,order_id_list)
    support_data,freq_transaction_1=create_freq_transaction(dataset,order_id_list,cl,min_support=min_support)
    all_freq_transaction=[freq_transaction_1]
    while len(all_freq_transaction[-1])>0:
        
        ck=create_ck(all_freq_transaction[-1])
        support_data_k,freq_transaction_k=create_freq_transaction(dataset,order_id_list,ck,min_support=min_support)
        support_data.update(support_data_k)
        all_freq_transaction.append(freq_transaction_k)
    return support_data,all_freq_transaction    

In [19]:
s,fs=Apriori(zl_order_multi_order_drop_index,order_id_unique,min_support=0.5)

In [20]:
s

{frozenset({'lsku'}): 0.998531939721696}

In [21]:
fs

[[frozenset({'lsku'})], []]